Input File Creation
===========
First let's start with some tools to create input files for a given deployment schedule.

In [10]:
import json
from math import ceil
from copy import deepcopy

import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

In [22]:
with open('once-through.json') as f:
    BASE_SIM = json.load(f)
DURATION = BASE_SIM['simulation']['control']['duration']
YEARS = ceil(DURATION / 12)
MONTH_SHUFFLE = (0, 6, 9, 3, 7, 5, 11, 1, 4, 8, 10, 2)
NULL_SCHEDULE = {'build_times': [{'val': 1}], 
                 'n_build': [{'val': 0}], 
                 'prototypes': [{'val': 'LWR'}]}
LWR_PROTOTYPE = {'val': 'LWR'}

In [20]:
BASE_SIM['simulation']['region']['institution']['config']['DeployInst']

{'build_times': [{'val': 1}],
 'n_build': [{'val': 0}],
 'prototypes': [{'val': 'LWR'}]}

In [26]:
def deploy_inst_schedule(Theta):
    if np.sum(Theta) == 0: 
        return NULL_SCHEDULE
    sched = {}
    build_times = sched['build_times'] = []
    n_build = sched['n_build'] = []
    prototypes = sched['prototypes'] = []
    m = 0
    for i, theta in enumerate(Theta):
        if theta <= 0:
            continue
        build_times.append(i*12 + MONTH_SHUFFLE[m])
        n_build.append(theta)
        prototypes.append(LWR_PROTOTYPE)
        m = (m + 1) % 12
    return sched

def make_sim(Theta, fname='sim.json'):
    sim = deepcopy(BASE_SIM)
    inst = sim['simulation']['region']['institution']
    inst['config']['DeployInst'] = deploy_inst_schedule(Theta)
    with open(fname, 'w') as f:
        json.dump(sim, f)
    return sim

In [27]:
s = make_sim([0, 1])

In [28]:
s['simulation']['region']['institution']['config']['DeployInst']

{'build_times': [12], 'n_build': [1], 'prototypes': [{'val': 'LWR'}]}